# Name: Mohammad Javad Noroozi
# Student Number: 99102434
# Project

### import the necessary library

In [1]:
# !pip install numpy

In [2]:
import json
import datetime
# import numpy as np
from collections import Counter


In [3]:
# For measuring the total running time of this notebook
start_notebook_t = datetime.datetime.now()
print(start_notebook_t)

2024-02-05 16:12:18.528141


In [4]:
import findspark
findspark.find()
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("HW1") \
    .master("local[*]") \
    .getOrCreate()

sc=spark.sparkContext


24/02/05 16:12:19 WARN Utils: Your hostname, javad-IdeaPad-Gaming-3-15IAH7 resolves to a loopback address: 127.0.1.1; using 192.168.137.8 instead (on interface enp48s0)
24/02/05 16:12:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/05 16:12:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Read data

In [5]:
tweets_rdd = sc.textFile("twitter_data_v2.jsonl")


### Downsampling (If your RAM memory is low)

In [6]:
tweets_rdd = tweets_rdd.sample(False, 0.01, seed=42)


### Show a sample of twitts

In [7]:
# tweets_rdd.takeSample(False, 1, 19)

### Show size of data (No. tweets)

In [8]:
# tweets_rdd.count()

### Extract all user IDs

In [9]:
# users = tweets_rdd.map(lambda x: json.loads(x)["user"].get('id')).collect()

### Select a User

In [10]:
# '''
# you must choose user id in this section.
# you can change # in `counts.most_common()[#][0]` for this purpose.
# '''
# counts = Counter(users)
# q_uid = counts.most_common()[50][0]
# q_uid = '1365385186364567553'
# print('query user id:', q_uid)

## Extracting the selected user's favorite tweet ids

### Define Parameters for the Primary Rating.

In [11]:
# Define Parameters
owner_s = 1

# following parameters should not be equal to owner_s
rep_to_s = 0.2
rep_from_s = 0.9

ret_to_s = 0.5
ret_from_s = 1.5

qut_to_s = 0.6
qut_from_s = 1.5

# the Value that the Tweet Vdds to the User
VTAU = {
    'origin': .1,
    'quoter': .1,  # user qoute a tweet by this tweet
    'quoted_by': 0.99,  # user qouted by this tweet
    'retweeter': .1,  # user retweet a tweet by this tweet
    'retweeted_by': 0.9,  # user retweeted by this tweet
    'replier': .1,  # user reply a tweet by this tweet
    'replied_by': 0.89,  # user replied by this tweet
}

tweet_necessary_info = [
    "tweet_type",
    "view_count",
    "twitter_engagement_rate",
    "retweet_count",
    "reply_count",
    "reach",
    "quote_count",
    "influence",
    "impression_rate",
    "impression",
    "favorited",
    "favorite_count",
    "engagement_rate",
    "engagement",
]

user_necessary_info = [
    "verified",
    "followers_count",
    "friends_count",
    "listed_count",
    "favourites_count",
    "statuses_count",
]

### Define functions for primary and secondary ratings

In [12]:
def extract_u_t_tuples(x: str) -> list:

    tweet_info = json.loads(x)
    tweet_statistics = dict(filter(
        lambda y: y[0] in tweet_necessary_info,
        tweet_info.items()
        )
    )

    t_value = calculate_tweet_value(tweet_statistics)
    user_statistics = dict(filter(
        lambda y: y[0] in user_necessary_info,
        tweet_info['user'].items()
        )
    )
    u1_value = calculate_user_value(user_info=user_statistics)
    
    tweet_value = interpolate_t_u_v0(t_value, u1_value)
    u1 = tweet_info["user"].get('id')

    if tweet_info.get('quoted_status', {}).get('user'):
        u2 = tweet_info.get('quoted_status', {}).get('user', {}).get('id')
        return [
            (u1, [VTAU['quoter'] * tweet_value]),
            (u2, [VTAU['quoted_by'] * tweet_value]),
        ]
    elif tweet_info.get('retweeted_status', {}).get('user'):
        u2 = tweet_info.get('retweeted_status', {}).get('user', {}).get('id')
        return [
            (u1, [VTAU['retweeter'] * tweet_value]),
            (u2, [VTAU['retweeted_by'] * tweet_value]),
        ]
    elif tweet_info.get('in_reply_to_user_id_str', None):
        u2 = tweet_info.get('in_reply_to_user_id_str')
        return [
            (u1, [VTAU['replier'] * tweet_value]),
            (u2, [VTAU['replied_by'] * tweet_value]),
        ]

    return [
        (u1, [VTAU['origin'] * tweet_value]),
    ]


def interpolate_t_u_v0(t_value, u_value):
    return 0.5*t_value + 0.5*u_value


def extract_t_u_tuples(x: str):
    '''
    input: tweet's json (a string)
    output: (t, (tv, uv, {uid})
    '''
    tweet_info = json.loads(x)


    tweet_statistics = dict(filter(
        lambda y: y[0] in tweet_necessary_info,
        tweet_info.items()
        )
    )
    tid = tweet_info['id']
    tweet_value = calculate_tweet_value(tweet_statistics)

    user_statistics = dict(filter(
        lambda y: y[0] in user_necessary_info,
        tweet_info['user'].items()
        )
    )
    u1_value = calculate_user_value(user_info=user_statistics)

    u1_id = tweet_info["user"].get('id')


    origin_tid = [
            tweet_info.get('in_reply_to_status_id_str', None),
            tweet_info.get('quoted_status', {}).get('id'),
            tweet_info.get('retweeted_status', {}).get('id'),
        ]
    origin_tid = [i for i in origin_tid if i is not None]

    if not origin_tid:
        return [(tid, (interpolate_t_u_values(tweet_value, u1_value), {u1_id}))]

    return [
        (tid, (interpolate_t_u_values(tweet_value, u1_value), {u1_id})),
        (origin_tid[0], (None, {u1_id})),
    ]


def interpolate_t_u_values(t_value, u_value):
    return 0.8*t_value + 0.2*u_value


def calculate_user_value_based_on_tweets(tweets_type_value: tuple) -> tuple:
    '''
    tweets_type_value: (u, [t_value, ...])
    output: (u, value)
    '''
    u, tv = tweets_type_value
    if len(tv) < 40:
        return(u, 0)
    u_value = avg(sorted(tv, key=lambda x: -x)[:5])
    return (u, u_value)


def calculate_user_value(user_info: dict) -> float:
    '''
    user_info: user information
    [
        "verified",
        "followers_count",
        "friends_count",
        "listed_count",
        "favourites_count",
        "statuses_count",
    ]

    output: a value betweet 0 and 1
    '''
    values = []
    # values.append(int(user_info['verified']))
    values.append(min(user_info['statuses_count']/max(user_info['favourites_count'], 1), 1))
    values.append(min(user_info['followers_count']/1000, 1))

    return 0.15 * avg(values) + 0.85 * int(user_info['verified'])


def calculate_tweet_value(tweet_statistics: dict) -> float:
    '''
    tweet_statistics has following keys:
    [
        "tweet_type",
        "view_count",
        "twitter_engagement_rate",
        "retweet_count",
        "reply_count",
        "reach",
        "quote_count",
        "influence",
        "impression_rate",
        "impression",
        "favorited",
        "favorite_count",
        "engagement_rate",
        "engagement",
    ]
    output: a value betweet 0 and 1
    '''
    values = []
    # values.append(int(tweet_statistics['favorited']))
    values.append(min(tweet_statistics['engagement_rate'], 1)/100*20)
    values.append(min(tweet_statistics['impression_rate'], 1)/100*5)
    values.append(min(tweet_statistics['influence'], 1)*10)
    values.append(min(tweet_statistics['twitter_engagement_rate'], 1)*20)
    vc = max(tweet_statistics['view_count'], 1)
    values.append(min(5*tweet_statistics['favorite_count']/vc, 1))
    values.append(min(10*tweet_statistics['retweet_count']/vc, 1))
    values.append(min(5*tweet_statistics['reply_count']/vc, 1)*5)
    values.append(min(10*tweet_statistics['quote_count']/vc, 1)*100)
    veiw_coef = min(vc/1000, 1)**3
    return avg(sorted(values)[:])*0.5*veiw_coef/70 + int(tweet_statistics['favorited'])*0.5


def calculate_tweet_value_based_on_users(t_users_value: tuple) -> tuple:
    '''
    users_value: (t, (tv, {uid})
    UCFD: User Credential Factor Dictionary
    {uid: credential}

    output: (t, value)
    '''
    UCFD = calculate_tweet_value_based_on_users.UCFD
    t, values = t_users_value
    tv, uids = values

    u_values = [UCFD[uid] for uid in uids]
    
    u_value = avg(sorted(u_values, key=lambda x: -x)[:5])
    t_value = interpolate_t_u_values2(tv, u_value)
    return (t, t_value)


def interpolate_t_u_values2(t_value, u_value):
    return 0.5 * t_value + 0.5 * u_value


def avg(lst): 
    return sum(lst) / len(lst) 

In [13]:
# [(u, [(t_type, t_value)]), ...]
user_tid_score_rdd = tweets_rdd.flatMap(extract_u_t_tuples)

# [(u, [(t_type, t_value), ...]), ...]
user_tids_scores_rdd = user_tid_score_rdd.reduceByKey(
    lambda x, y: x if not x.extend(y) else x)


user_credential_factor_rdd = user_tids_scores_rdd.map(calculate_user_value_based_on_tweets)


user_credential_factor_dict = dict(user_credential_factor_rdd.collect())

In [14]:
tweets_filtered_rdd = tweets_rdd.filter(lambda x: json.loads(x)['tweet_type'] == 'generated')

# [(t, (tv, {uid})] or [(t, (None, None, {uid})]
tweet_user_value_rdd = tweets_filtered_rdd.flatMap(extract_t_u_tuples)

# [(t, (tv, {uid})]
tweet_users_value_rdd = tweet_user_value_rdd.reduceByKey(
    lambda x, y: (x[0], (x[1][0], x[1][1].union(y[1][1]))) if x[1][0] else\
    (y[0], (y[1][0], y[1][1].union(x[1][1])))
)

# [(t, value)]
calculate_tweet_value_based_on_users.UCFD = user_credential_factor_dict
tweet_value_rdd = tweet_users_value_rdd.map(calculate_tweet_value_based_on_users)

In [15]:
th = 0.05
valid_tweets = tweet_value_rdd.filter(lambda x: x[1] > th)\
                                .take(1000)

In [16]:
len(valid_tweets)

293

In [17]:
necessary_keys = ['tweet_type', 'user', 'id','text', 'name', 'screen_name', 
                'in_reply_to_status_id_str', 'in_reply_to_user_id_str',
                'quoted_status', 'retweeted_status','profile_image_url',
                "tweet_type",
                "view_count",
                "twitter_engagement_rate",
                "retweet_count",
                "reply_count",
                "reach",
                "quote_count",
                "influence",
                "impression_rate",
                "impression",
                "favorited",
                "favorite_count",
                "engagement_rate",
                "engagement",
                 ]

valid_tweets_id, _ = zip(*valid_tweets) 
valid_tweets = tweets_rdd.filter(lambda x: json.loads(x)['id'] in valid_tweets_id and not json.loads(x)['user']['verified'])\
                             .map(lambda x: dict(filter(lambda y: y[0] in necessary_keys, json.loads(x).items())))\
                             .collect()
valid_tweets2 = valid_tweets.copy()


In [18]:
def show_tweets(tweets, tweets_type):
    user_statistics_keys = [
        "name",
        "verified",
        "followers_count",
        "friends_count",
        "listed_count",
        "favourites_count",
        "statuses_count",
    ]

    tweet_statistics_keys = [
        "tweet_type",
        "view_count",
        "twitter_engagement_rate",
        "retweet_count",
        "reply_count",
        "reach",
        "quote_count",
        "influence",
        "impression_rate",
        "impression",
        "favorited",
        "favorite_count",
        "engagement_rate",
        "engagement",
    ]
    print('-'*100)
    print(f'Number of {tweets_type}:', len(tweets))
    ts = [t['text'].replace('\u200c', '').split('\n') for t in tweets]
    for i, t in enumerate(ts):
        print('\n'*2)
        print('_'*100)
        user_statistics = dict(filter(
            lambda y: y[0] in user_statistics_keys,
            tweets[i]['user'].items()
            )
        )
        print(user_statistics)

        tweet_statistics = dict(filter(
            lambda y: y[0] in tweet_statistics_keys,
            tweets[i].items()
            )
        )
        print(tweet_statistics)
        print('_'*100)
        for line in t:
            print(line)
    print('-'*100)


### Separate tweets based on tweet_type and display them

In [19]:
show_tweets(valid_tweets, 'validate_tweets')

----------------------------------------------------------------------------------------------------
Number of validate_tweets: 12



____________________________________________________________________________________________________
{'name': '👑Shayan👊♂️', 'verified': False, 'followers_count': 38757, 'friends_count': 4445, 'listed_count': 104, 'favourites_count': 160610, 'statuses_count': 63297}
{'retweet_count': 239, 'reply_count': 11, 'quote_count': 2, 'favorite_count': 1264, 'favorited': False, 'view_count': 7155, 'tweet_type': 'generated', 'engagement': 1514, 'impression': 7155, 'influence': 0.0391, 'impression_rate': 0.113, 'engagement_rate': 0.0239, 'twitter_engagement_rate': 21.16}
____________________________________________________________________________________________________
آمد! به ما یادآوری کرد که کی هستیم و رفت!💔
#مجیدرضا_رهنورد https://t.co/Uez7E3D67W



____________________________________________________________________________________________________
{'name': 'جوا